# The Littlest JupyterHub による軽量Python実習環境の構築

[JupyterHub](https://jupyter.org/hub) は、Webブラウザからアクセス可能なマルチユーザ対応の認証機能付きJupyterNotebookサーバです。

JupyterHubを利用して管理者が用意したNotebookをユーザがブラウザからすぐに実行可能な環境を提供できるため、Pythonによるプログラミング研修やワークショップを開催したり、講義演習環境として活用したりするのに適しています。

本ハンズオンでは、JupyterHubを小規模なグループで手軽に利用することを想定し、単一のサーバで実行するために開発された「[The Littlest JupyterHub](https://tljh.jupyter.org/)」（以下 “TLJH” と略）をVCPを用いて構築します。  
ハンズオンご参加の皆様には、このテンプレートでTLJHによるVCPアプリケーション環境を構築していただきます。

![](images/403-overview.png)

## 構築環境情報の入力
TLJH環境の構築情報を入力します。必要に応じ、下記の情報を修正してください。

**★ハンズオンでは以下のパラメータを変更しないでください★**

In [ ]:
####################################################
### ハンズオンでは以下のパラメータを変更しないでください。###
####################################################

# UnitGroup名
ugroup_name = 'handson403'

# プロバイダ
vc_provider = 'aws'

# フレーバー
vcnode_flavor = 'small'

# VCノードに付与する固定IPアドレス
fixed_ipaddress = '172.30.2.201'

# VCノードのディスクサイズ (単位: GB)
volume_size = 32

## VCノードの作成

はじめに、アプリケーションVCP を利用するために必要なアクセストークンを入力し、VCP SDK を初期化します。

### アクセストークンの入力

OCSハンズオン用トークンはこちら→ [token.txt](../../../files/token.txt)

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

### VCP の初期化
VCP を初期化します。エラーになった場合、この章のセルを `unfreeze` してから、もう一度アクセストークンを入力してください。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(vcc_access_token)

### TLJH を実行するVCノードの作成

クラウド上のインスタンスをVCノードとして作成します。

 ### VCノードのspecを指定する
 
TLJH を利用するのに十分な性能・容量のノードspecを指定します。  
固定割当IPアドレスは、ハンズオン環境のNAT Proxyサーバに予め設定されているIPアドレスを使います。

In [ ]:
# UnitGroup の作成
unit_group = vcp.create_ugroup(ugroup_name)

# VCノード spec
spec = vcp.get_spec(vc_provider, vcnode_flavor)

# spec オプション (ディスクサイズ 単位:GB)
spec.volume_size = volume_size

# spec オプション (固定割当IPアドレス)
spec.ip_addresses = [fixed_ipaddress]

# ssh keyfiles
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')
spec.set_ssh_pubkey(ssh_public_key)

### Unitの作成とVCノードの起動
Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。処理が完了するまで1分半～2分程度かかります。

In [ ]:
# Unitの作成（同時に VCノードが作成される）
unit = unit_group.create_unit('tljh-node', spec)

### 疎通確認

まず、ssh の `known_hosts` の設定を行います。

その後、VCノードに対して`uname -a`を実行し、`ubuntu x86_64 Linux` が起動していることを確認します。起動していない場合は、`spec.image` に誤りがあります。本テンプレート下部にある「環境の削除」を実行、 `spec.image` を修正、全てのセルを `unfreeze` してから、最初から再実行してください。

In [ ]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す
print(ip_address)

# ssh 設定
!touch ~/.ssh/known_hosts
!ssh-keygen -R {ip_address}    # ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts    # ホストキーの登録

# システムの確認
!ssh {ip_address} uname -a

## TLJH (The Littlest JupyterHub) 環境の構築

VCノード上に、本ハンズオン用に用意したThe Littlest JupyterHubのコンテナイメージを使用して環境を構築します。

### TLJHコンテナイメージの取得

VCノード上にコンテナイメージを取得するために `docker pull` を実行します。

In [ ]:
docker_image = 'public.ecr.aws/niivcp/vcp/tljh:handson2206'

# イメージの取得
!ssh {ip_address} /usr/local/bin/docker pull {docker_image}

# イメージの確認
!ssh {ip_address} /usr/local/bin/docker images

### TLJHコンテナの起動

pullしたイメージを使ってTLJHコンテナを起動します。

In [ ]:
# コンテナ実行 (docker run)
!ssh {ip_address} /usr/local/bin/docker run -d --privileged --net=host --name tljh {docker_image}

# 起動状態の確認 (docker ps)
!ssh {ip_address} /usr/local/bin/docker ps

### TLJH 管理者ユーザ (teacher) の作成

TLJH に管理者としてログインするためにアカウントを作成します。

- 本ハンズオンでは、管理者ユーザを `admin-user01` という名前で作成します。
- TLJHコンテナ上で `tljh-config` コマンドを実行することでユーザ追加だけでなく他の様々な設定を行うことができます。
- cf. <https://tljh.jupyter.org/en/latest/topic/tljh-config.html>

In [ ]:
# admin-user01 を管理者ユーザとして追加
!ssh {ip_address} /usr/local/bin/docker exec tljh \
    tljh-config add-item users.admin admin-user01

# 現在の設定内容を確認
!ssh {ip_address} /usr/local/bin/docker exec tljh \
    tljh-config show

実行中のTLJHに設定内容を反映するために `tljh-config reload` コマンドを実行します。

In [ ]:
!ssh {ip_address} /usr/local/bin/docker exec tljh \
    tljh-config reload

#### TLJH の Web インターフェースのカスタマイズ

今、このノートブックを実行しているJupyterNotebook環境と、構築したTLJHのJupyterHubのWebインターフェースを識別しやすくするために、TLJH側にカスタマイズしたCSSファイルを設定します。

以下のセルを実行します。

In [ ]:
!scp ./403/custom.css  {ip_address}:/tmp/
!ssh {ip_address} /usr/local/bin/docker exec tljh mkdir -p /etc/skel/.jupyter/custom
!ssh {ip_address} /usr/local/bin/docker cp /tmp/custom.css tljh:/etc/skel/.jupyter/custom/custom.css

## TLJH の管理と利用

VCノード上に起動したTLJHにブラウザからアクセスし、Python実習環境として必要な作業を行います。

### 管理者ユーザとしてブラウザからログイン

以下のセルを実行し、ログイン先URLを作成します。

In [ ]:
vcc_ctr = vcp.vcc_info()['host']
print("https://{}/".format(vcc_ctr.split(':')[0]))

#### ログインパスワード設定

初回ログイン時に入力したパスワードが以降のログインパスワードとして設定されます。

![](images/403-tljh-login.png)

#### 初期画面

ログイン後の初期画面は、ファイルやディレクトリが1つもない状態となっています。

![](images/403-tljh-filebrowser.png)

### 一般ユーザ (student) アカウントの追加

管理者用コントロールパネルの機能を利用し、管理者以外のユーザを追加します。

#### 画面右上の **Control Panel** ボタンを押す

![](images/403-tljh-control_1.png)

#### **Admin** メニューを選択する

![](images/403-tljh-control_2.png)

#### **Add Users** ボタンを押す

![](images/403-tljh-control_3.png)

#### アカウント名を入力して **Add Users** ボタンを押す

- アカウント名は改行して複数入力することが可能

![](images/403-tljh-addusers.png)

### Python パッケージの追加

Python実習環境として利用するために、ライブラリ・パッケージの追加を試します。

- cf. <https://tljh.jupyter.org/en/latest/howto/env/user-environment.html#installing-pip-packages>

<p>
    
ここでは以下の [pip](https://pypi.org/project/pip/) パッケージをインストールします。

- [numpy](https://numpy.org/)
- [matplotlib](https://matplotlib.org/)
- [pandas](https://pandas.pydata.org/)


#### 管理者ユーザとしてログインしてTerminalを開く

![](images/403-tljh-terminal.png)

#### Terminalで以下のコマンドを実行する

```
sudo -E pip install numpy matplotlib pandas
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh {ip_address} /usr/local/bin/docker exec -u jupyter-admin-user01 tljh \
  sudo -E /opt/tljh/user/bin/pip install numpy matplotlib pandas

#### インストールできたことを確認する

```
sudo -E pip list | grep -e numpy -e matplotlib -e pandas
```

  - 正常にインストールできていれば、以下のようにバージョン情報が出力される。

    ```
    matplotlib             3.5.2
    matplotlib-inline      0.1.3
    numpy                  1.22.4
    pandas                 1.4.2
    ```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh {ip_address} /usr/local/bin/docker exec -u jupyter-admin-user01 tljh \
  sudo -E /opt/tljh/user/bin/pip list | grep -e numpy -e matplotlib -e pandas

### 教材Notebookの配布（ファイル共有）

実習用に用意された教材がある場合、TLJH環境の各ユーザから読み込み専用の共有フォルダにアクセスさせることで教材配布が行えるようにします。

- cf. <https://tljh.jupyter.org/en/latest/howto/content/share-data.html#option-2-create-a-read-only-shared-folder-for-data>

#### 管理者ユーザのTerminalで共有フォルダを作成する

- 管理者ユーザとしてログインし、Terminal からコマンドを実行して共有フォルダを作成します。
- 共有フォルダは `/srv/data/` 以下であれば任意のフォルダ名が使用できます。

```
sudo mkdir -p /srv/data/shared
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh {ip_address} /usr/local/bin/docker exec tljh mkdir -p /srv/data/shared

#### ユーザのホームディレクトリに共有フォルダへのリンクを作成する

- `/etc/skel/` ディレクトリに共有フォルダへのシンボリックリンクを作成します。  
  これにより、新規に追加したユーザのホームディレクトリに共有フォルダが表示されます。
  * ユーザ追加後、一度もログインしたことがないユーザは初回ログイン時に共有フォルダが表示されます。
- 管理者ユーザとしてログインし、 Terminalから以下のコマンドを実行します。

```
sudo ln -s /srv/data/shared /etc/skel/
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
!ssh {ip_address} /usr/local/bin/docker exec tljh ln -sf /srv/data/shared /etc/skel/

#### TLJH環境の共有フォルダにファイルを配置する

本ハンズオン向けにサンプルのNotebookファイル [sample-notebook.ipynb](./403/sample-notebook.ipynb) を用意したので、これをJupyterHubにコピーします。

以下のセルを実行することで、ここからVCノードを経由してTLJHコンテナ上の共有フォルダにコピーを行います。

In [ ]:
filename = 'sample-notebook.ipynb'

# このNotebookからVCノードへのコピー
!scp ./403/{filename}  {ip_address}:/tmp/{filename}

# VCノードからTLJHコンテナへのコピー
!ssh {ip_address} /usr/local/bin/docker cp /tmp/{filename} tljh:/srv/data/shared/{filename}

### 一般ユーザとしてログインし、共有フォルダを開く

一般ユーザとしてTLJHにログインすると、ホームディレクトリに共有フォルダが表示されます。

> 管理者ユーザとしてログイン中の場合、一度ログアウトする必要があります。

In [ ]:
# ログイン先URL生成用
vcc_ctr = vcp.vcc_info()['host']
print("https://{}/".format(vcc_ctr.split(':')[0]))

## (参考) 応用編

ここでは、TLJH の運用環境をさらにカスタマイズするためのいくつかの方法を紹介します。

### Jupyter Notebook 拡張機能の導入

そのままでも十分使いやすいJupyter Notebookですが、 `jupyter_contrib_nbextensions` というライブラリを導入することで様々な拡張機能が使えるようになり、より便利になります。

- cf. <https://tljh.jupyter.org/en/latest/howto/admin/enable-extensions.html>

#### 管理者ユーザのTerminalでコマンド実行

管理者ユーザとしてログインし、Terminal から以下のコマンドを実行します。

1.  pip パッケージをインストールする

```
sudo -E pip install jupyter_contrib_nbextensions==0.5.1
```

2. 拡張機能のスタイルファイルをJupyter環境に追加する

```
sudo -E jupyter contrib nbextension install --sys-prefix
```

3. [Table of Contents (2)](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/toc2/README.html) （目次表示） の拡張機能を有効化する

```
sudo -E jupyter nbextension enable toc2/main --sys-prefix
```

4. 現在有効な拡張機能を確認する

```
jupyter nbextension list
```

Terminal を使用せずに、以下のセルでも同じコマンド操作が可能です。

In [ ]:
# パッケージのインストール
!ssh {ip_address} /usr/local/bin/docker exec -u jupyter-admin-user01 tljh \
  sudo -E /opt/tljh/user/bin/pip install jupyter_contrib_nbextensions==0.5.1

# 拡張機能の追加
!ssh {ip_address} /usr/local/bin/docker exec -u jupyter-admin-user01 tljh \
  sudo -E /opt/tljh/user/bin/jupyter contrib nbextension install --sys-prefix

# 拡張機能の有効化
!ssh {ip_address} /usr/local/bin/docker exec -u jupyter-admin-user01 tljh \
  sudo -E /opt/tljh/user/bin/jupyter nbextension enable toc2/main --sys-prefix

# 現在有効な拡張機能の確認
!ssh {ip_address} /usr/local/bin/docker exec -u jupyter-admin-user01 tljh \
  /opt/tljh/user/bin/jupyter nbextension list

### 各ユーザのNotebookサーバのリソース制限

各ユーザが使用できる最大メモリ量やCPUを管理者が `tljh-config` コマンドで設定して制限することができます。

- cf. <https://tljh.jupyter.org/en/latest/topic/tljh-config.html#user-server-limits>

#### メモリ制限

個々のユーザが使用可能な最大メモリ量を指定します。  
管理者ユーザとしてログインし、Terminal から以下のコマンドを実行します。

```
sudo tljh-config set limits.memory 1G
```

- `None` を指定すると、メモリ制限は無効化される。

#### CPU制限

各ユーザが使用可能なCPUコアの合計を指定します。  
管理者ユーザとしてログインし、Terminal から以下のコマンドを実行します。

```
sudo tljh-config set limits.cpu 2
```

- `2` の場合、2個のCPUをフルに使用できることを表す。 `0.5` は、1個のCPUの半分を表す。

#### リソース制限設定の適用

実行中のTLJH環境に設定を反映させるためには、管理者ユーザのTerminalから以下のコマンドを実行します。

1. 確認

```
sudo tljh-config show
```

2. 適用

```
sudo tljh-config reload
```

## 環境の削除

ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。  
この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

In [ ]:
unit_group.cleanup()

削除後の状態を確認します。

In [ ]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

In [ ]:
# UnitGroup強制削除
# UnitGroup作成後、エラーが発生するなど強制的に削除する必要が生じた場合のみ、コメントを外して利用します。
# ugroup = vcp.get_ugroup('handson403')
# ugroup.cleanup()